In [25]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
from scipy import signal
import pyedflib
from pyedflib import highlevel
import os
import h5py
import torch
import torchvision

In [26]:
#Constants 
dataPath = r"C:\Users\markd\Documents\GitHub\NAThacks\data\files"
h5Path = r"h5py/1personval2.h5"
numChannels = 64
labels = {
    "T0" : "rest",
    "T1" : ["leftHand", "bothHands"],
    "T2" : ["rightHand", "bothFeet"]
}
labelsDict = {
    "rest": 0,
    "leftHand": 1,
    "rightHand": 2,
    "bothHands": 3,
    "bothFeet": 4
}
frequency = 160
time_period = 640
prc_overlap = .90
chunk_size = 1

In [27]:
rest = []
leftHand = []
rightHand = []
bothHands = []
bothFeet = []

In [28]:
with h5py.File(h5Path, "w") as specs:
    specs.create_dataset("rest",(chunk_size,81,31,64), maxshape=(None,81,31,64),compression="gzip")
    specs.create_dataset("leftHand",(chunk_size,81,31,64), maxshape=(None,81,31,64), compression="gzip")
    specs.create_dataset("rightHand",(chunk_size,81,31,64), maxshape=(None,81,31,64), compression="gzip")
    specs.create_dataset("bothHands",(chunk_size,81,31,64), maxshape=(None,81,31,64), compression="gzip")
    specs.create_dataset("bothFeet",(chunk_size,81,31,64), maxshape=(None,81,31,64), compression="gzip")
    

def storeData(data, dataset : str, size):
    with h5py.File(h5Path, "r+") as f:
        df = f[dataset]
        current_shape = df.shape
        df.resize((current_shape[0] + size, 81, 31, 64))
        df[current_shape[0]:, :] = data
        print(f"{dataset} adjusted to {current_shape[0] + size}")

In [29]:
with h5py.File(h5Path, "w") as specs:
    specs.create_dataset("Samples",(0,81,31,64), maxshape=(None,81,31,64),compression="gzip")
    specs.create_dataset("Classes",(0,5), maxshape=(None,5),compression="gzip")
    
    print(f"h5 made, stores: Samples - {specs['Samples'].shape} | Classes - {specs['Classes'].shape}")
    print(f"{specs['Samples'][:]}")
    print(f"{specs['Classes'][:]}")
    # specs["Samples"][:] = np.delete(specs["Samples"], axis=0, obj=[0])
    # specs["Classes"][:] = np.delete(specs["Classes"], axis=0, obj=[0])
    # print(f"updated to Samples - {specs['Samples'].shape} | Classes - {specs['Classes'].shape}")

h5 made, stores: Samples - (0, 81, 31, 64) | Classes - (0, 5)
[]
[]


In [30]:
def storeDataSingle(data, dataset : str, size):
    with h5py.File(h5Path, "r+") as f:
        df = f['Samples']
        current_shape = df.shape
        print(f"Current shape is {df.shape}! Class to be added is: {dataset}")
        df.resize((current_shape[0] + size, 81, 31, 64))
        df[current_shape[0]:, :] = data
        print(f"Samples adjusted to {current_shape[0] + size} after adding {size}")

        # Store class label
        # Classes need to be one-hot encoded
        class_idx = labelsDict[dataset]
        label = np.zeros([size, 5])
        label[:,class_idx] = 1

        labels = f['Classes']
        current_shape = labels.shape
        labels.resize((current_shape[0] + size, 5))
        labels[current_shape[0]:, :] = label


In [31]:
#Generating spectograms Images
person = 1
for folder in os.listdir(dataPath)[69:70]:
    for fileName in os.listdir(os.path.join(dataPath,folder)):

        #Skipping files .event files
        if fileName.find("event") != -1:
            continue

        #reading EDF file and extracting data
        filePath = os.path.join(dataPath,folder,fileName)
        annotations = ""
        file = ""

        #Getting the annotations and Data
        file = pyedflib.EdfReader(filePath)
        annotations = file.readAnnotations()
        file.close()

        signals, signal_headers, header = highlevel.read_edf(filePath)
        
        count = 0
        
        #Looping through the various 
        for i, period in enumerate(annotations[0]):
            #Getting signal data for 4 second period
            data = signals[:,int(annotations[0][i]):int(annotations[0][i]+time_period)]

            f, t, image = signal.spectrogram(data,frequency, nperseg = frequency, noverlap = frequency * prc_overlap)
            image = np.transpose(image,(1,2,0))

            #Assigning label to file
            label = annotations[2][i]

            #Categories of the different tests
            leftOrRight = ["03","04","07","08","11","12"]
            if label == "T1":
                if any(x == fileName[5:7] for x in leftOrRight):
                    leftHand.append(image)
                    if len(leftHand) == chunk_size:
                        storeDataSingle(leftHand,"leftHand", chunk_size)
                        leftHand = []
                else: 
                    bothHands.append(image)
                    if len(bothHands) == chunk_size:
                        storeDataSingle(bothHands,"bothHands", chunk_size)
                        bothHands = []
            elif label == "T2":
                if any(x == fileName[5:7] for x in leftOrRight):
                    rightHand.append(image)
                    if len(rightHand) == chunk_size:
                        storeDataSingle(rightHand,"rightHand", chunk_size)
                        rightHand = []
                else:
                    bothFeet.append(image)
                    if len(bothFeet) == chunk_size:
                        storeDataSingle(bothFeet,"bothFeet", chunk_size)
                        bothFeet = []
            else:
                rest.append(image)
                if len(rest) == chunk_size:
                        storeDataSingle(rest,"rest", chunk_size)
                        rest = []
            count += 1
    print(f"person {person} completed")
    person +=1

Current shape is (0, 81, 31, 64)! Class to be added is: rest
Samples adjusted to 1 after adding 1
Current shape is (1, 81, 31, 64)! Class to be added is: rest
Samples adjusted to 2 after adding 1
Current shape is (2, 81, 31, 64)! Class to be added is: rest
Samples adjusted to 3 after adding 1
Current shape is (3, 81, 31, 64)! Class to be added is: leftHand
Samples adjusted to 4 after adding 1
Current shape is (4, 81, 31, 64)! Class to be added is: rest
Samples adjusted to 5 after adding 1
Current shape is (5, 81, 31, 64)! Class to be added is: rightHand
Samples adjusted to 6 after adding 1
Current shape is (6, 81, 31, 64)! Class to be added is: rest
Samples adjusted to 7 after adding 1
Current shape is (7, 81, 31, 64)! Class to be added is: rightHand
Samples adjusted to 8 after adding 1
Current shape is (8, 81, 31, 64)! Class to be added is: rest
Samples adjusted to 9 after adding 1
Current shape is (9, 81, 31, 64)! Class to be added is: leftHand
Samples adjusted to 10 after adding 1
C

In [37]:
storeDataSingle(rest,"rest", len(rest))
rest = []
storeDataSingle(bothFeet,"bothFeet", len(bothFeet))
bothFeet = []
storeDataSingle(leftHand,"leftHand", len(leftHand))
leftHand = []
storeDataSingle(rightHand, "rightHand",len(rightHand))
rightHand = []
storeDataSingle(bothHands,"bothHands",len(bothHands))
bothHands = []

Current shape is (362, 81, 31, 64)! Class to be added is: rest
Samples adjusted to 362 after adding 0
Current shape is (362, 81, 31, 64)! Class to be added is: bothFeet
Samples adjusted to 362 after adding 0
Current shape is (362, 81, 31, 64)! Class to be added is: leftHand
Samples adjusted to 362 after adding 0
Current shape is (362, 81, 31, 64)! Class to be added is: rightHand
Samples adjusted to 362 after adding 0
Current shape is (362, 81, 31, 64)! Class to be added is: bothHands
Samples adjusted to 362 after adding 0
